In [19]:
%cd ..

/home/jovyan/work/boxes_vis/git/obstacle-detection


In [16]:
import sys
sys.path.append('/home/jovyan/work/boxes_vis/git/obstacle-detection/')
sys.path.append('/home/jovyan/work/obstacle-detection/model/')

In [17]:
sys.path

['../',
 '../../',
 '/home/jovyan/work/boxes_vis/git/obstacle-detection/examples',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/home/jovyan/.ipython',
 '/home/jovyan/work/boxes_vis/git/obstacle-detection/',
 '..',
 '/home/jovyan/work/boxes_vis/git/obstacle-detection/',
 '/home/jovyan/work/obstacle-detection/model/']

In [6]:
import yaml
import numpy as np
import pandas as pd
import pyntcloud


num ='000'
scan = np.fromfile('../dataset/sequences/00/velodyne/000' + num + '.bin', dtype=np.float32)
scan = scan.reshape((-1, 4))

label = np.fromfile('../dataset/sequences/00/labels/000' + num + '.label', dtype=np.uint32)
label = label.reshape((-1))

In [8]:
with open('../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
obstacle_dct = config['obstacles']

In [10]:
# seg id -> seg name
obstacle_dct

{10: 'car',
 11: 'bicycle',
 13: 'bus',
 15: 'motorcycle',
 16: 'on-rails',
 18: 'truck',
 20: 'other-vehicle',
 30: 'person',
 31: 'bicyclist',
 32: 'motorcyclist',
 252: 'moving-car',
 253: 'moving-bicyclist',
 254: 'moving-person',
 255: 'moving-motorcyclist',
 256: 'moving-on-rails',
 257: 'moving-bus',
 258: 'moving-truck',
 259: 'moving-other-vehicle'}

In [11]:
leaning_map_obstacles = config['learning_map']

In [12]:
# ground truth id -> learning id
# leaning_map_obstacles

In [13]:
mapped_obstacles_lst = list(set([leaning_map_obstacles[id] for id in obstacle_dct.keys()]))
mapped_obstacles_lst

[1, 2, 3, 4, 5, 6, 7, 8]

In [21]:
import sys
sys.path.append("..")
from model.tasks.semantic.infer2 import get_user 

### Load Model

In [22]:
#user = get_user("dataset", "darknet21")
user = get_user("darknet21")

----------
INTERFACE:
model darknet21
----------

Opening arch config file from darknet21
[Errno 2] No such file or directory: '/home/jovyan/work/obstacle-detection/model/darknet21/arch_cfg.yaml'
Error opening arch yaml file.


NameError: name 'quit' is not defined

In [73]:
%timeit predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.986173152923584 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.015446186065674 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9522764682769775 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9182658195495605 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.86429762840271 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9911413192749023 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.4889438152313232 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.6889333724975586 sec
3.21 s ± 311 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.8645498752593994 sec


In [66]:
predicted_labels.shape

(124668,)

### Visualize segmentated point cloud

In [75]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-1.4353549480438232, 45.90348565578461, 3.02…

In [79]:
pcloud.points.seg_id.unique()

array([16, 15, 13, 19, 18, 14,  1,  8,  9, 10, 17, 11,  5,  7])

### Visualize only obstacles

In [89]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(2.0857174396514893, 45.77106475830078, 1.971…

### Load Model

In [81]:
#user = get_user("dataset", "darknet21")
user = get_user( "squeezeseg")

----------
INTERFACE:
model squeezeseg
----------

Opening arch config file from squeezeseg
Opening data config file from squeezeseg
model folder exists! Using model from squeezeseg
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  915540
Total number of parameters requires_grad:  915540
Param encoder  724032
Param decoder  179968
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [82]:
%timeit predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.7203648090362549 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.9078242778778076 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.8646385669708252 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.8509914875030518 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.7517774105072021 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.7306826114654541 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.7433125972747803 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.679023027420044 sec
865 ms ± 77.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6660304069519043 sec


In [84]:
predicted_labels.shape

(124668,)

### Visualize segmentated point cloud

In [85]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-1.4353549480438232, 45.90348565578461, 3.02…

In [86]:
pcloud.points.seg_id.unique()

array([18, 16, 19, 13, 15,  4,  5, 12,  1,  6,  8, 14, 17,  7, 10,  9, 11,
        3,  2])

### Visualize only obstacles

In [90]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(2.0857174396514893, 45.77106475830078, 1.971…